In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import pyodbc
import matplotlib.pyplot as plt 
from pandas import ExcelWriter
from IPython.display import display, HTML
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 35)

In [2]:
# Open connection to Netezza databases for direct SQL queries
dsn_name = "DSN=Netezza;"
conn = pyodbc.connect(dsn_name)

In [3]:
#SQL Query to identify all cases refunded from 2018-11-01 and 2018-11-30
case_numbers_sql = """
SELECT
    icare.CASENUMBER AS "Case Number",
    SUBSTR(icare.LONG_MTCN__C, LENGTH(icare.LONG_MTCN__C) - 9, 10) AS MTCN, 
    icare.REFUND_MTCN__C AS "Refund MTCN",
    icare.REFUND_AMOUNT__C AS "Refund Amount",
    icare.REFUND_CURRENCY__C AS "Refund Currency",
    icare.COUNTRY__C AS "Send Country",
    icare.DESTINATION_COUNTRY__C AS "Receive Country",
    icare.REMEDIATION__C AS "Refund Type (Fee or FX)",
    transaction_info.RECPRODCT AS "Product Code",
    icare.SUBJECT AS Issue

FROM SERVICE_CLOUD_STG.DBAUSER.SFDC_CASE icare

INNER JOIN (
            SELECT fintx.RECPRODCT, trandetail.MTCN03 || trandetail.MTCN07 AS MTCN, fintx.RECISODTE
            FROM TDB.ETL_TDB.DB100PDLIB_FNTXKP01 fintx
            INNER JOIN TDB.ETL_TDB.DW400PDLIB_DWTXKP01 trandetail on fintx.TXNKEY = trandetail.TXNKEY
            ) 
                AS transaction_info ON SUBSTR(icare.LONG_MTCN__C, LENGTH(icare.LONG_MTCN__C) -9, 10) = transaction_info.MTCN
                    AND SUBSTR(icare.MT_SEND_DATE__C,1,10) = SUBSTR(transaction_info.RECISODTE,1,10)

WHERE 
    icare.RECORD_TYPE_NAME__C = 'Remediation'
    AND icare.DATE_TIME_FIRST_CLOSED__C BETWEEN '2019-01-01' AND '2019-01-31'
"""

#SQL query to obtain transaction information from TDB
currency_conversion_sql = f"""
SELECT pcurr.CURR, MIN(tran.RATEPAYEXC)/100 as MINRATE, MAX(tran.RATEPAYEXC)/100 AS MAXRATE, AVG(tran.RATEPAYEXC)/100 AS MEANRATE, median(tran.RATEPAYEXC)/100 AS MEDIAN

FROM TDB.ETL_TDB.DB100PDLIB_FNTXKP01 fin

INNER JOIN TDB.ETL_TDB.DB100PDLIB_DBCURP01 pcurr ON fin.PAYCURKEY = pcurr.CURKEY
INNER JOIN TDB.ETL_TDB.DW400PDLIB_DWTXKP01 tran on fin.TXNKEY = tran.TXNKEY

WHERE fin.PAYISODTE BETWEEN CURRENT_DATE - 10 AND CURRENT_DATE - 2
GROUP BY pcurr.CURR
"""

In [4]:
#Execute SQL statements
case_numbers = pd.read_sql_query(case_numbers_sql,conn)
currency_conversion = pd.read_sql_query(currency_conversion_sql,conn)

In [5]:
case_numbers = case_numbers.drop_duplicates(subset = 'MTCN', keep = 'first')

In [6]:
#Create additional column to review differences between median and mode currency rates
currency_conversion['median_check_perc_diff'] = (currency_conversion['MEDIAN'] - currency_conversion['MEANRATE'])/ currency_conversion['MEDIAN']
currency_conversion['median_mode_diff'] = (currency_conversion['MEDIAN'] - currency_conversion['MEANRATE']) 

In [7]:
#Create function to review currency conversion
#Section is still a work in progress not sure how to append each identified issue to a new dataframe
currency_conversion_columns = currency_conversion.columns
investigate_currency_conversion = pd.DataFrame(columns = [currency_conversion_columns])

def currency_review(row):
    if row['median_mode_diff']/row['MEDIAN'] and row['median_mode_diff']/row['MINRATE'] and row['median_mode_diff']/row['MAXRATE']  > 0.01:
        investigate_currency_conversion.append(row)
    else:
        pass

In [8]:
#Create dataframe listing any currency conversion rates which should be further investigated
#currency_conversion.apply(lambda row: currency_review(row), axis = 1)

In [9]:
#Merge case_numbers dataframe and mtcn_details dataframe on column MTCN
mtcn_details_currency_conversion_merged = pd.merge(case_numbers, currency_conversion, how = 'left', left_on = 'Refund Currency', right_on = 'CURR')

In [10]:
#Check length of new dataframe matches original dataframe list to confirm no MTCN's were lost
if mtcn_details_currency_conversion_merged.shape[0] == case_numbers.shape[0]:
    print('All MTCNs appear to be included')
else:
    print('Error: MTCNs appear to be missing')

All MTCNs appear to be included


In [11]:
#Calculate refund amount in USD and drop columns which are not required
mtcn_details_currency_conversion_merged['Amount USD'] = round(mtcn_details_currency_conversion_merged['Refund Amount'] / mtcn_details_currency_conversion_merged['MEDIAN'],2)
mtcn_details_currency_conversion_merged = mtcn_details_currency_conversion_merged.drop(['CURR','MINRATE','MAXRATE','MEANRATE','MEDIAN','median_check_perc_diff', 'median_mode_diff'], axis = 1)
mtcn_details_currency_conversion_merged = mtcn_details_currency_conversion_merged.sort_values(by = 'Amount USD', ascending = False)

In [15]:
#Export Report in USD
writer = ExcelWriter(r'C:\Users\309270\Desktop\portfolio_analytics\accounting report\january_2019_account_report.xlsx')
mtcn_details_currency_conversion_merged.to_excel(writer,'data', index =False)
writer.save()